In [17]:
import torch
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
torch.cuda.set_device(0)
torch.cuda.reset_max_memory_allocated()

In [7]:
from ultralytics import YOLO
from ray import tune

# Load the model
model = YOLO("yolov8n.pt")

# Hyperparameter Tuning with Ray Tune:
result = model.tune(
    data='E:/files_main/projects_ml/hard-hat-detection/Hard-Hat-Workers-8/custom_hard_hat.yaml',
    gpu_per_trial=1,
    max_samples=5,
    space={
      "lr0": tune.uniform(1e-5, 1e-1),
      "weight_decay": tune.uniform(0.0, 0.001),
      "momentum": tune.uniform(0.6, 0.7),
      "degrees": tune.uniform(0.0, 30.0)
      },
    train_args={
      "epochs": 10, 
      "imgsz":640, 
      "batch":32
      }
)

https://docs.ultralytics.com/tasks/detect/#val
https://docs.ultralytics.com/modes/predict/


2023-05-13 15:22:52,396	WARNING worker.py:1986 -- Warning: The actor ImplicitFunc is very large (12 MiB). Check that its definition is not implicitly capturing a large array or other object in scope. Tip: use ray.put() to put large objects in the Ray object store.
2023-05-13 15:22:52,461	WARNING util.py:244 -- The `start_trial` operation took 0.551 s, which may be a performance bottleneck.
(_tune pid=12148) Ultralytics YOLOv8.0.99  Python-3.10.11 torch-2.0.1+cu118 CUDA:0 (NVIDIA GeForce RTX 2060, 6144MiB)
(_tune pid=12148) yolo\engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=E:/files_main/projects_ml/hard-hat-detection/Hard-Hat-Workers-8/custom_hard_hat.yaml, epochs=10, patience=50, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, overlap

Trial name,date,done,epoch,hostname,iterations_since_restore,metrics/mAP50(B),metrics/mAP50-95(B),metrics/precision(B),metrics/recall(B),node_ip,pid,time_since_restore,time_this_iter_s,time_total_s,timestamp,training_iteration,trial_id
_tune_f69d2_00000,2023-05-13_15-43-46,False,9,DaveJammin_TUF,11,0.967544,0.642616,0.95016,0.911061,127.0.0.1,12148,1241.48,24.9535,1241.48,1683963826,11,f69d2_00000


(_tune pid=12148) 
(_tune pid=12148)       Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
  0%|          | 0/154 [00:00<?, ?it/s]
       2/10       4.3G      1.503      1.384      1.231        123        640:   1%|          | 1/154 [00:00<01:00,  2.52it/s]
       2/10       4.3G       1.49      1.268      1.213        179        640:   1%|▏         | 2/154 [00:00<00:58,  2.61it/s]
       2/10       4.3G      1.449      1.257      1.213        167        640:   2%|▏         | 3/154 [00:01<00:57,  2.63it/s]
       2/10       4.3G      1.464      1.226      1.233        164        640:   3%|▎         | 4/154 [00:01<00:56,  2.65it/s]
       2/10       4.3G      1.448      1.212      1.238        132        640:   3%|▎         | 5/154 [00:01<00:56,  2.65it/s]
       2/10       4.3G      1.447       1.24      1.238        186        640:   4%|▍         | 6/154 [00:02<00:56,  2.63it/s]
       2/10       4.3G      1.434      1.216      1.243        192        640:   5%

In [6]:
result.get_best_result(metric='metrics/mAP50(B)', mode='max')

Result(
  metrics={'metrics/precision(B)': 0.9630100273724719, 'metrics/recall(B)': 0.9384104267825198, 'metrics/mAP50(B)': 0.9819171830901267, 'metrics/mAP50-95(B)': 0.6788961687706643, 'epoch': 9, 'done': True, 'trial_id': '69709_00003', 'experiment_tag': '3_degrees=7.8162,lr0=0.0047,momentum=0.6247,weight_decay=0.0004'},
  path='e:\\files_main\\projects_ml\\hard-hat-detection\\src\\runs\\_tune_2023-05-13_13-17-09\\_tune_69709_00003_3_degrees=7.8162,lr0=0.0047,momentum=0.6247,weight_decay=0.0004_2023-05-13_14-34-11',
  checkpoint=None
)

In [ ]:
# Load the model.
model = YOLO('yolov8n.pt')
 
# Training.
results = model.train(
   data='../Hard-Hat-Workers-8/custom_hard_hat.yaml',
   imgsz=640,
   epochs=10,
   batch=8,
   name='yolov8n_custom'
   )